In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Conv2D, MaxPool2D , Activation , Dropout , Flatten , Dense
from keras.preprocessing.image import ImageDataGenerator, img_to_array , load_img
from glob import glob

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Loading and Visualization of Data

In [ ]:
test_path= pd.read_csv("/kaggle/input/mnist-in-csv/mnist_test.csv")
train_path = pd.read_csv("/kaggle/input/mnist-in-csv/mnist_train.csv")

In [ ]:
train_path.shape

In [ ]:
test_path.shape

In [ ]:
train_path.head()

In [ ]:
train_path["label"].unique()

In [ ]:
test_path.head()

In [ ]:
y_train=train_path["label"]
x_train=train_path.drop(columns ="label")
y_test=test_path["label"]
x_test=test_path.drop(columns ="label")

In [ ]:
x_train.head()

In [ ]:
plt.figure(figsize=(10,5))
g= sns.countplot(y_train, palette="icefire")
plt.title("Number of digit classes")
plt.show()


In [ ]:
y_train.shape

In [ ]:
x_train.shape

# Reshape

In [ ]:
x_train= x_train.values.reshape(-1,28 ,28,1)
x_test=x_test.values.reshape(-1, 28,28,1)
print("x_train shape:" , x_train.shape)
print("x_test shape:" , x_test.shape)


# Normalization

In [ ]:
x_train= x_train/255.0
test_path= test_path/255

# Label Encoding

In [ ]:
from keras.utils.np_utils import to_categorical
y_train= to_categorical(y_train, num_classes=10)

# Visualization

In [ ]:
image_index=777
print(y_train[image_index])
plt.imshow(x_train[image_index], cmap='Greys')


In [ ]:
a=img_to_array(x_train[image_index])
print(a.shape)

In [ ]:
className= glob("train_path+'/*'")
numberOfClass=len(className)
print(numberOfClass)

# Train Test Split

In [ ]:
x_train, x_val,y_train, y_val = train_test_split(x_train, y_train ,test_size= 0.2, random_state=42) 
print("x train size:" ,x_train.shape)
print("x validation: ",x_val.shape)
print("y train size: ",y_train.shape)
print("y validation size:" ,y_val.shape)

# Create Model

In [ ]:
model= Sequential()

model.add(Conv2D(32 ,(3,3), activation="relu",input_shape=a.shape))
model.add(MaxPool2D())
model.add(Dropout(0.2))
model.add(Conv2D(32,(3,3),activation="relu"))
model.add(MaxPool2D())
model.add(Conv2D(64,(3,3), activation="relu"))


model.add(Flatten())
model.add(Dense(250, activation="relu"))
model.add(Dropout(0.5))
model.add(Dense(10, activation="softmax"))

model.compile(loss="categorical_crossentropy",
              optimizer="rmsprop",
              metrics=["accuracy"])
batch_size=32
epoch= 20

# Data Generator

In [ ]:
datgen= ImageDataGenerator(shear_range=0.3,
                          horizontal_flip=True,
                          width_shift_range=0.5,
                          zoom_range=0.3)

datgen.fit(x_train)

In [ ]:
history=model.fit_generator(datgen.flow(x_train,y_train, batch_size=batch_size),epochs=epoch, validation_data=(x_val,y_val), steps_per_epoch=x_train.shape[0]//batch_size)

# Results

In [ ]:
plt.plot(history.history["loss"], label="Train loss")
plt.plot(history.history["val_loss"], label="Validation Loss")
plt.legend()
plt.show()

In [ ]:
plt.plot(history.history["accuracy"], label="Train Accuracy")
plt.plot(history.history["val_accuracy"], label="Validation Accuracy")
plt.legend()
plt.show()